In [3]:
import pandas as pd
data = pd.read_csv('./1-4.csv',encoding='utf-8')#gb18030
print(data.shape)
data.head(1)

(2656, 5)


,order_id,user_name,content,Unnamed: 3,Unnamed: 4
0,1,搜狐,实际死亡或超90万!福奇承认美疫情被“严重低估”,28分钟前,搜狐


In [4]:
data.describe()

,order_id
count,2656.000000
mean,1328.500000
std,766.865481
min,1.000000
25%,664.750000
50%,1328.500000
75%,1992.250000
max,2656.000000


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2656 entries, 0 to 2655
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   order_id    2656 non-null   int64 
 1   user_name   2656 non-null   object
 2   content     2656 non-null   object
 3   Unnamed: 3  2656 non-null   object
 4   Unnamed: 4  2656 non-null   object
dtypes: int64(1), object(4)
memory usage: 103.9+ KB


In [6]:
# 复制一个pandas的处理副本
data_clean = data.copy()

In [7]:
# 加载用户评论向量化模块
from sklearn.feature_extraction.text import CountVectorizer
# 对测试集和训练集进行划分
from sklearn.model_selection import train_test_split
import numpy as np
import jieba

In [8]:
%time data['text_cut'] = data['content'].apply(lambda x: " ".join(jieba.lcut(str(x))))
data.head(1)

Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\sievier\AppData\Local\Temp\jieba.cache
Loading model cost 1.002 seconds.
Prefix dict has been built successfully.


Wall time: 1.5 s


,order_id,user_name,content,Unnamed: 3,Unnamed: 4,text_cut
0,1,搜狐,实际死亡或超90万!福奇承认美疫情被“严重低估”,28分钟前,搜狐,实际 死亡 或超 90 万 ! 福奇 承认 美 疫情 被 “ 严重 低估 ”


In [9]:
# 调整低词频带来的影响
vectorizer = CountVectorizer(token_pattern='\[?\w+\]?', max_features=5000)
vectorizer.fit_transform(data['text_cut'])

<2656x4755 sparse matrix of type '<class 'numpy.int64'>'
	with 28539 stored elements in Compressed Sparse Row format>

In [10]:
vectorizer.vocabulary_

{'实际': 1887,
 '死亡': 3004,
 '或超': 2359,
 '90': 200,
 '万': 331,
 '福奇': 3553,
 '承认': 2417,
 '美': 3792,
 '疫情': 3405,
 '被': 4016,
 '严重': 482,
 '低估': 771,
 '求真': 3053,
 '观察': 4043,
 '丨': 487,
 '之下': 563,
 '华商': 1223,
 '太难': 1750,
 '了': 593,
 '3': 95,
 '人': 653,
 '编造': 3773,
 '涉外': 3165,
 '虚假': 3988,
 '信息': 835,
 '获刑': 3960,
 '印度': 1261,
 '将': 1941,
 '加剧': 1139,
 '全球': 914,
 '供应': 806,
 '短缺': 3513,
 '中国': 503,
 '为': 534,
 '拉美': 2459,
 '防控': 4535,
 '提供': 2532,
 '有力': 2841,
 '支持': 2555,
 '无惧': 2686,
 '高烧': 4727,
 '难退': 4593,
 '欧佩克': 2977,
 '继续': 3764,
 '看好': 3475,
 '今年': 691,
 '原油': 1288,
 '需求': 4615,
 '复苏': 1654,
 '赶上': 4203,
 '无望': 2690,
 '失控': 1754,
 '重创': 4452,
 '手机': 2379,
 '产业': 636,
 '供应链': 807,
 '转移': 4260,
 '计划': 4072,
 '落空': 3974,
 '总理': 2262,
 '莫迪': 3954,
 '确定': 3531,
 '不会': 385,
 '亲自': 652,
 '出席': 1055,
 'g7': 227,
 '高峰会': 4723,
 '受': 1347,
 '影响': 2185,
 '富士康': 1919,
 '工厂': 2010,
 '的': 3447,
 'iphone': 234,
 '12': 21,
 '产量': 641,
 '降': 4558,
 '一半': 280,
 '绝望': 3760,
 '又': 1300,
 '无奈

In [13]:
# 加载词频统计模块
from sklearn.feature_extraction.text import CountVectorizer

# 设定 topn 的高频词语阈值，可以根据需要进行改变大小
topn = 400
# 加载统计模块
count_extract = CountVectorizer(max_features=topn)
# 开始对语料进行统计,对每一家店铺生成一个词云的图片
count_extract.fit_transform(data['text_cut'][1:400])

# 输入数据中的每一个词为（word，value）这样的元祖形式，然后将所有的词放入一个list中
comment_count_pair = [(k, int(v))
                      for k, v in count_extract.vocabulary_.items()]

# 加载词云模块
from pyecharts.charts import WordCloud
# 加载 pyecharts 配置模块配置词云显示
from pyecharts import options as opts

comment_wordcloud = WordCloud()
# word_size_range 表示词语大小范围，可以根据需要改变
comment_wordcloud.add(
    series_name="", data_pair=comment_count_pair, word_size_range=[1, 100]
)
# 设置词云的全局配置
comment_wordcloud.set_global_opts(
    title_opts=opts.TitleOpts(title="高频词云分析",
                              # 设置标题文字大小
                              title_textstyle_opts=opts.TextStyleOpts(font_size=15)),
    # 显示词云中的词频数字
    tooltip_opts=opts.TooltipOpts(is_show=True),
)
# 直接在 notebook 中显示词云
comment_wordcloud.render_notebook()
